World Development Indicators - worldbank.org <br/>
Fabio Cardoso - Dec/2021 <br/><br/>

Datasets: <br/><br/>

Country Indicators: <br/>
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators <br/>
Note: dataset updated with the topics presented in https://data.worldbank.org/indicator?tab=all - Utils notebook. <br/><br/>

Country Codes ISO 3166: <br/>
https://www.iban.com/country-codes <br/><br/>

Note: <br/>
For some orphans indicators, the topic 'private-sector' was assumed - Utils notebook. <br/>
. Firms visited or required meetings with tax officials (% of firms) <br/>
. Net ODA provided to the least developed countries (% of GNI) <br/>
. Net ODA provided, to the least developed countries (current US )−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡2015𝑈𝑆) <br/>
. Net ODA provided, total (current US$)

In [1]:
# Parameters

cloud = True

In [2]:
# Imports

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import zlib
import pickle
from threading import Thread
from multiprocessing import Pool
import time
from datetime import datetime

In [ ]:
# The code was removed by Watson Studio for sharing.

In [3]:
# Storage instantiation

if cloud:
    import types
    import pandas as pd
    import ibm_boto3
    from botocore.client import Config

    def __iter__(self): return 0

    cnx_fcsinsights = ibm_boto3.client(
        service_name = 's3',
        ibm_api_key_id = parm_ibm_api_key_id,
        ibm_service_instance_id = parm_ibm_service_instance_id,
        ibm_auth_endpoint = parm_ibm_auth_endpoint,
        config = Config(signature_version='oauth'),
        endpoint_url = parm_endpoint_url
    )

bkt_in = parm_bkt_in
bkt_out = parm_bkt_out

key_in = 'WDI treated-step2.csv.gz'
key_out = 'WDI treated-step3.csv.gz'

In [4]:
# Read data

if cloud:
    cnx_fcsinsights.download_file(Bucket=bkt_in, Key=key_in, Filename=key_in)
    df = pd.read_csv(key_in, compression='gzip')
else:
    df = pd.read_csv('F:/WDI Working Files/'+key_in, compression='gzip')

In [5]:
# Cleansing worker 

py = """import numpy as np
import pandas as pd
import zlib
import pickle

def outliers_treatment(params):

    cont = 0
    part = params.get('part')
    
    f = open('thread_'+str(part)+'.txt', mode='w')
    f.write(str(part)+' '+str(cont))
    f.close()
                
    df_part = pickle.loads(zlib.decompress(params.get('dfz_part')))
    
    for country_code in df_part['Country Code'].unique():
    
        df_country = df_part[df_part['Country Code']==country_code]
        
        for indicator_code in df_country['Indicator Code'].unique():

            df_indicator = df_country[(df_country['Indicator Code']==indicator_code)].copy()

            q1 = df_indicator['Value'].quantile(0.25)
            q3 = df_indicator['Value'].quantile(0.75)
            iqr = q3 - q1  
            lim_inf = q1 - 1.5 * iqr
            lim_sup = q3 + 1.5 * iqr

            df_indicator['Value'] = df_indicator['Value'].apply(lambda v: lim_inf if v < lim_inf else lim_sup if v > lim_sup else v)

            ixs = df_indicator.index
            
            df_part.at[ixs, 'Value'] = df_indicator['Value'].values
                
            cont+=1
            if cont%500==0:
                f = open('thread_'+str(part)+'.txt', mode='w')
                f.write(str(part)+' '+str(cont))
                f.close()
                
    dfz_part = zlib.compress(pickle.dumps(df_part))
    del df_part
    return dfz_part"""

f = open('outliers_treatment3.py', mode='w',encoding='utf8')
f.write(py)
f.close()
import outliers_treatment3

In [6]:
# Compressing parts

df = df.reset_index().drop(columns=['index'])
dfz_part1 = zlib.compress(pickle.dumps(df.query("Part==1")))
dfz_part2 = zlib.compress(pickle.dumps(df.query("Part==2")))
dfz_part3 = zlib.compress(pickle.dumps(df.query("Part==3")))
dfz_part4 = zlib.compress(pickle.dumps(df.query("Part==4")))

In [7]:
# Parallel processing

p = Pool(processes = 4)

lst_params = []
lst_params.append({'part':1, 'dfz_part':dfz_part1})
lst_params.append({'part':2, 'dfz_part':dfz_part2})
lst_params.append({'part':3, 'dfz_part':dfz_part3})
lst_params.append({'part':4, 'dfz_part':dfz_part4})

def thread_unica(parm, lst_params):
    
    print('acionando thread de processors')
    ret = p.map(outliers_treatment3.outliers_treatment, lst_params)
    parm.append(ret)

parm = []
thread = Thread(target=thread_unica, args=(parm, lst_params))
thread.start()

acionando thread de processors


In [8]:
# Following processing

s1_ant = s2_ant = s3_ant = s4_ant = -1
s1 = s2 = s3 = s4 = -1

frozen=False
time.sleep(30)
while not frozen:
    
    try:
        f1=open('thread_1.txt',mode='r')
        s1=f1.read()
        f1.close()
    except: pass
    print(s1)
    
    try:
        f2=open('thread_2.txt',mode='r')
        s2=f2.read()
        f2.close()
    except: pass
    print(s2)
    
    try:
        f3=open('thread_3.txt',mode='r')
        s3=f3.read()
        f3.close()
    except: pass    
    print(s3)
    
    try:
        f4=open('thread_4.txt',mode='r')
        s4=f4.read()
        f4.close()
    except: pass    
    print(s4)   
    
    if s1==s1_ant and s2==s2_ant and s3==s3_ant and s4==s4_ant:
        frozen=True
        break
    print()
    
    s1_ant = s1
    s2_ant = s2
    s3_ant = s3
    s4_ant = s4
    
    time.sleep(30)

1 500
2 500
3 500
4 500

1 1000
2 1000
3 1000
4 1000

1 1500
2 1500
3 1500
4 1500

1 2000
2 2000
3 2000
4 2000

1 3000
2 3000
3 3000
4 3000

1 3500
2 3500
3 3500
4 3500

1 4000
2 4000
3 4000
4 4000

1 4500
2 4500
3 5000
4 5000

1 5500
2 5500
3 5500
4 5500

1 6000
2 6000
3 6000
4 6000

1 6500
2 6500
3 6500
4 6500

1 7000
2 7000
3 7500
4 7500

1 8000
2 8000
3 8000
4 8000

1 8500
2 8500
3 8500
4 8500

1 9000
2 9000
3 9000
4 9000

1 10000
2 10000
3 10000
4 10000

1 10500
2 10500
3 10500
4 10500

1 11000
2 11000
3 11000
4 11000

1 11500
2 11500
3 12000
4 12000

1 12500
2 12500
3 12500
4 12500

1 13000
2 13000
3 13000
4 13000

1 13500
2 13500
3 13500
4 13500

1 14500
2 14000
3 14500
4 14500

1 15000
2 15000
3 15000
4 15000

1 15500
2 15500
3 15500
4 15500

1 16500
2 16000
3 16000
4 16000

1 17000
2 17000
3 17000
4 17000

1 17500
2 17500
3 17500
4 17500

1 18000
2 18000
3 18000
4 18000

1 19000
2 18500
3 19000
4 19000

1 19500
2 19500
3 19500
4 19500

1 20000
2 20000
3 20000
4 20000

1 20500


In [9]:
# Consolidating results

for i in range(4):
    print(i)
    parm[0][i] = pickle.loads(zlib.decompress(parm[0][i]))

df = pd.concat([parm[0][0], parm[0][1], parm[0][2], parm[0][3]], axis=0)
print("ok")

0
1
2
3
ok


In [10]:
# Save intermediate result

if cloud:
    df.to_csv(key_out, compression='gzip', index=False)
    cnx_fcsinsights.upload_file(Bucket=bkt_out, Key=key_out, Filename=key_out)
else:
    df.to_csv('F:/WDI Working Files/'+key_out, compression='gzip', index=False)

In [11]:
print('ok')

ok
